In [14]:
# imports:
import urllib.request
import csv
import os
import itertools
import logging
import json
import numpy as np
import gensim
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from gensim.utils import smart_open, simple_preprocess
from gensim.corpora.wikicorpus import _extract_pages, filter_wiki
from gensim.parsing.preprocessing import STOPWORDS
from xml.etree.cElementTree import iterparse
from sklearn.cluster import KMeans
km=None

In [2]:

# List Categories:
category_list = ["Mathematics","Technology","Music","History","Geography","Arts","Health","Nature","Religion","Literature"]
testFolder =  "./Simplex1/test/"
root_folder = "./Simplex1/"
list.sort(category_list)
root_folder='./'+''.join([x[0] for x in category_list])+'/'

if not os.path.exists(root_folder):
    os.mkdir(root_folder)

wiki_bow_path = root_folder+'wiki_bow.mm'

In [3]:
# Download Page Ids:
#https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&categories=mathematics&doit=Do it!
for cat in category_list:
    url="https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories="+cat
    print(url)
    urllib.request.urlretrieve(url, root_folder+cat+".csv")

    print(cat+".csv")
    

https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories=Arts
Arts.csv
https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories=Geography
Geography.csv
https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories=Health
Health.csv
https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories=History
History.csv
https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories=Literature
Literature.csv
https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories=Mathematics
Mathematics.csv
https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories=Music
Music.csv
https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories=Nature
Nature.csv
https://pets

In [4]:
# CSV to XML Download data:

def getData(ids,outputFile):
    url="https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=xml&pageids="+ids
    req = urllib.request.urlopen(url)
    if req.getcode() == 200:
        soup = BeautifulSoup(req.read(), 'html.parser')
        s = soup.find_all('page')
        for si in s:
            outputFile.write(str(si))

def batchTrain(file):
    outputFile = open(file.replace(".csv",".xml"), 'a', encoding="utf8")
    outputFile.write("<pages>")
                    
    csvReader = csv.reader(open(file,'r'))
    totalRecords = sum(1 for row in csv.reader(open(file,'r',encoding="UTF-8")) )
    print (totalRecords)
    start = 0
    end = start + 50
   
    while (start <= totalRecords):
        pageIds = ""
        for row in itertools.islice(csv.reader(open(file,'r',encoding="UTF-8")),start,end):
            pageIds = pageIds + row[2] + "|"
        
        getData(pageIds,outputFile)
        start = end + 1
        end = start + 50
        if end> totalRecords:
            end=totalRecords
            
    outputFile.write("</pages>")
    

for path, subdirs, files in os.walk(root_folder):
    for file in files:
        if file.endswith('.csv'):
            if not os.path.exists(root_folder+file.replace(".csv",".xml")):
                batchTrain(path + file)

print ("Done")



Done


In [5]:
# Train Model
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

def head(stream, n=10):
    """Convenience fnc: return the first `n` elements of the stream, as plain list."""
    return list(itertools.islice(stream, n))

def my_extract_pages(f):
    elems = (elem for _, elem in iterparse(f, events=("end",)))
    page_tag = "rev"
    for elem in elems:
        if elem.tag == page_tag and elem.text != None:
            text = elem.text
            yield text
            elem.clear()

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

def iter_wiki(dump_file):
    """Yield each article from the Wikipedia dump, as a `(title, tokens)` 2-tuple."""
    ignore_namespaces = 'Wikipedia Category File Portal Template MediaWiki User Help Book Draft'.split()
    for text in my_extract_pages(smart_open(dump_file)):
        text = filter_wiki(text)
        tokens = tokenize(text)
        if len(tokens) < 50:
            continue  # ignore short articles and various meta-articles
        yield tokens
        

In [6]:

#print(id2word_wiki)


In [7]:
class WikiCorpus(object):
    def __init__(self, dump_file, dictionary, clip_docs=None):
        """
        Parse the first `clip_docs` Wikipedia documents from file `dump_file`.
        Yield each document in turn, as a list of tokens (unicode strings).
        
        """
        self.dump_file = dump_file
        self.dictionary = dictionary
        self.clip_docs = clip_docs
    
    def __iter__(self):
        for tokens in itertools.islice(iter_wiki(self.dump_file), self.clip_docs):
            yield self.dictionary.doc2bow(tokens)
    
    def __len__(self):
        return self.clip_docs
    
#if not os.path.exists(wiki_bow_path):
for path, subdirs, files in os.walk(root_folder):
    del subdirs[:]
    for file in files:
        if file.endswith('.xml'):
            doc_path = path + file
            print(doc_path)
            stream = iter_wiki(doc_path)
            for tokens in itertools.islice(iter_wiki(doc_path), 8):
                print (tokens[:10])
            doc_stream = (tokens for tokens in iter_wiki(doc_path))
            %time id2word_wiki = gensim.corpora.Dictionary(doc_stream)
            print(id2word_wiki)

id2word_wiki.filter_extremes(no_below=10, no_above=0.1)

# create a stream of bag-of-words vectors
wiki_corpus = WikiCorpus(doc_path, id2word_wiki)

#wiki_bow_path = root_folder+'wiki_bow.mm'
%time gensim.corpora.MmCorpus.serialize(wiki_bow_path, wiki_corpus)

mm_corpus = gensim.corpora.MmCorpus(wiki_bow_path)
print(mm_corpus)
clipped_corpus = gensim.utils.ClippedCorpus(mm_corpus, 4000) 
%time lda_model = gensim.models.LdaModel(clipped_corpus, num_topics=len(category_list), id2word=id2word_wiki, passes=10, alpha='asymmetric')


./AGHHLMMNRT/Arts.xml
['clockwise', 'upper', 'left', 'self', 'portrait', 'vincent', 'van', 'gogh', 'female', 'ancestor']
['johann', 'wolfgang', 'von', 'goethe', 'german', 'artist', 'known', 'works', 'poetry', 'drama']
['art', 'deco', 'simply', 'referred', 'deco', 'style', 'visual', 'arts', 'architecture', 'design']
['aesthetics', 'spelled', 'æsthetics', 'esthetics', 'known', 'greek', 'αισθητική', 'aisthētiké', 'branch', 'philosophy']
['bauhaus', 'dessau', 'walter', 'gropius', 'expressionist', 'monument', 'march', 'deadtypography', 'herbert', 'bayer']
['triumph', 'immaculate', 'paolo', 'matteis', 'church', 'sant', 'andrea', 'al', 'quirinale', 'designed']
['example', 'modern', 'cartoon', 'text', 'excerpted', 'cartoonist', 'greg', 'williams', 'wikipedia', 'article']
['crochet', 'process', 'creating', 'fabric', 'interlocking', 'loops', 'yarn', 'thread', 'strands', 'materials']


INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : built Dictionary(76020 unique tokens: ['pmcflex', 'ascetic', 'stada', 'ivana', 'minty']...) from 1388 documents (total 958585 corpus positions)
INFO : adding document #0 to Dictionary(0 unique tokens: [])


Wall time: 7.91 s
Dictionary(76020 unique tokens: ['pmcflex', 'ascetic', 'stada', 'ivana', 'minty']...)
./AGHHLMMNRT/Geography.xml
['acre', 'unit', 'land', 'area', 'imperial', 'customary', 'systems', 'defined', 'area', 'chain']
['map', 'oceania', 'country', 'codes', 'country', 'codes', 'short', 'alphabetic', 'numeric', 'geographical']
['cairn', 'human', 'pile', 'stack', 'stones', 'word', 'cairn', 'comes', 'plural', 'cairns']
['important', 'explorations', 'state', 'societies', 'chronological', 'order', 'exploration', 'explorer', 'northwest', 'african']
['furlong', 'measure', 'distance', 'imperial', 'units', 'customary', 'units', 'equal', 'eighth', 'mile']
['far', 'east', 'alternate', 'geographical', 'term', 'english', 'equivalents', 'languages', 'infobox', 'right']
['clusters', 'cholera', 'cases', 'london', 'epidemic', 'classical', 'case', 'human', 'geography', 'human']
['graticule', 'earth', 'sphere', 'ellipsoid', 'lines', 'pole', 'pole', 'lines', 'constant', 'longitude']


INFO : built Dictionary(55743 unique tokens: ['isenbruch', 'fage', 'isothermal', 'gunnislake', 'saada']...) from 978 documents (total 653601 corpus positions)


Wall time: 5.73 s
Dictionary(55743 unique tokens: ['isenbruch', 'fage', 'isothermal', 'gunnislake', 'saada']...)
./AGHHLMMNRT/Health.xml
['hearing', 'aid', 'assistive', 'technology', 'umbrella', 'term', 'includes', 'assistive', 'adaptive', 'rehabilitative']
['antibiotic', 'resistance', 'tests', 'bacteria', 'streaked', 'dishes', 'white', 'antibiotic', 'impregnated', 'disks']
['beryllium', 'chemical', 'element', 'symbol', 'atomic', 'number', 'relatively', 'rare', 'element', 'universe']
['biostatistics', 'application', 'statistics', 'wide', 'range', 'topics', 'biology', 'science', 'biostatistics', 'encompasses']
['condom', 'sheath', 'shaped', 'barrier', 'device', 'sexual', 'intercourse', 'reduce', 'probability', 'pregnancy']
['chlorine', 'chemical', 'element', 'symbol', 'cl', 'atomic', 'number', 'second', 'lightest', 'halogens']
['chromium', 'chemical', 'element', 'symbol', 'cr', 'atomic', 'number', 'element', 'group', 'steely']
['cadmium', 'chemical', 'element', 'symbol', 'cd', 'atomic',

...

Wall time: 10.3 s
Dictionary(70949 unique tokens: ['tren', 'nanyo', 'nanorod', 'bachem', 'consecutive']...)


INFO : PROGRESS: saving document #1000
INFO : saved 1967x8885 matrix, density=1.969% (344087/17476795)
INFO : saving MmCorpus index to ./AGHHLMMNRT/wiki_bow.mm.index
INFO : loaded corpus index from ./AGHHLMMNRT/wiki_bow.mm.index
INFO : initializing corpus reader from ./AGHHLMMNRT/wiki_bow.mm
INFO : accepted corpus with 1967 documents, 8885 features, 344087 non-zero entries
INFO : using asymmetric alpha [0.20349776650601445, 0.15460680268266819, 0.12465746387131386, 0.10442834231175278, 0.089848016434062955, 0.078840302634700543, 0.070235422324486096, 0.063324036444942805, 0.05765099744942407, 0.052910849340634232]
INFO : using symmetric eta at 0.00011254924029262802
INFO : using serial LDA version on this node


Wall time: 10.4 s
MmCorpus(1967 documents, 8885 features, 344087 non-zero entries)


INFO : running online LDA training, 10 topics, 10 passes over the supplied corpus of 1967 documents, updating model once every 1967 documents, evaluating perplexity every 1967 documents, iterating 50x with a convergence threshold of 0.001000
INFO : -9.805 per-word bound, 894.4 perplexity estimate based on a held-out corpus of 1967 documents with 662487 words
INFO : PROGRESS: pass 0, at document #1967/1967
INFO : topic #9 (0.053): 0.003*"yes" + 0.003*"architecture" + 0.002*"ontology" + 0.002*"lte" + 0.002*"learning" + 0.002*"park" + 0.002*"nuclear" + 0.002*"canada" + 0.002*"mm" + 0.001*"gas"
INFO : topic #8 (0.058): 0.002*"invented" + 0.002*"observatory" + 0.002*"html" + 0.002*"document" + 0.002*"mm" + 0.002*"file" + 0.002*"nuclear" + 0.002*"metadata" + 0.002*"bc" + 0.002*"canada"
INFO : topic #2 (0.125): 0.008*"nuclear" + 0.008*"observatory" + 0.003*"learning" + 0.002*"anti" + 0.002*"document" + 0.002*"ontology" + 0.002*"clock" + 0.001*"test" + 0.001*"library" + 0.001*"invented"
INFO :

Wall time: 3min 16s


In [8]:
def calculateCentroid(topic_docs):
    test_doc = [tokens for tokens in iter_wiki(topic_docs)]
    part = [lda_model[id2word_wiki.doc2bow(tokens)] for tokens in test_doc]
    
    topic_dic={}
    
    for i in range(len(category_list)):
        topic_dic[i]=0
        
    for doc in part:
        for p in doc:
            topic_dic[p[0]] += p[1]
    
    centroid = [(x, topic_dic[x]/len(part)) for x in range(len(category_list))]
    return centroid

    
centroids_dict={}
for path, subdirs, files in os.walk(root_folder):
    del subdirs[:]
    for file in files:
        if file.endswith('.xml'):
            doc_path = path + file
            print(doc_path)
            centroid = calculateCentroid(doc_path)
            centroids_dict[file.replace(".xml","")]=centroid
            
print(centroids_dict)

./AGHHLMMNRT/Arts.xml
./AGHHLMMNRT/Geography.xml
./AGHHLMMNRT/Health.xml
./AGHHLMMNRT/History.xml
./AGHHLMMNRT/Literature.xml
./AGHHLMMNRT/Mathematics.xml
./AGHHLMMNRT/Music.xml
./AGHHLMMNRT/Nature.xml
./AGHHLMMNRT/Religion.xml
./AGHHLMMNRT/Technology.xml
{'Geography': [(0, 0.011791355894419096), (1, 0.16658928858671304), (2, 0.15731707482932109), (3, 0.088883679464277068), (4, 0.062664567894465895), (5, 0.054968828868599405), (6, 0.21751491475480583), (7, 0.035238980939058522), (8, 0.15251178715669866), (9, 0.045332156509909204)], 'Nature': [(0, 0.0040957422234728742), (1, 0.11640268790950072), (2, 0.084708711991682437), (3, 0.01835855424713544), (4, 0.15853358485424762), (5, 0.11681042214667688), (6, 0.092161462426417082), (7, 0.22212030865860119), (8, 0.15192920565220186), (9, 0.028873547948347438)], 'Music': [(0, 0.037226605732226789), (1, 0.31380052708233969), (2, 0.088398329707307183), (3, 0.020226880116930832), (4, 0.19232875046545511), (5, 0.01833184195890895), (6, 0.0179251184

In [21]:
def get_values(l):
    #print (l)
    g=[0]*len(category_list)
    for i in l:
        g[i[0]]=i[1]
        
    return g



def calculate_kmeans(part):
    km = KMeans(n_clusters=len(category_list), init='k-means++', max_iter=500, n_init=1)
    km.fit(part)
    print(km.cluster_centers_)
    return km


for path, subdirs, files in os.walk(root_folder):
    del subdirs[:]
    part=[]
    for file in files:
        if file.endswith('.xml'):
            doc_path = path + file
            print(doc_path)
            test_doc = [tokens for tokens in iter_wiki(doc_path)]
            #print(lda_model[id2word_wiki.doc2bow(tokens))
            
            part+=[get_values(lda_model[id2word_wiki.doc2bow(tokens)]) for tokens in test_doc]
        
    
    km = calculate_kmeans(part)
    
            


./AGHHLMMNRT/Arts.xml
./AGHHLMMNRT/Geography.xml
./AGHHLMMNRT/Health.xml
./AGHHLMMNRT/History.xml
./AGHHLMMNRT/Literature.xml
./AGHHLMMNRT/Mathematics.xml
./AGHHLMMNRT/Music.xml
./AGHHLMMNRT/Nature.xml
./AGHHLMMNRT/Religion.xml
./AGHHLMMNRT/Technology.xml
[[ 0.01627921  0.11715731  0.03181314  0.05778004  0.52589895  0.0677934
   0.0176143   0.04995426  0.08883577  0.01908459]
 [ 0.59972617  0.10568627  0.00981675  0.06194827  0.05198937  0.03325015
   0.02216838  0.01454496  0.04479864  0.04712439]
 [ 0.01412895  0.40229812  0.08994092  0.0183581   0.04869675  0.01734975
   0.0231164   0.14560316  0.16802676  0.06691448]
 [ 0.01318473  0.13624423  0.07369427  0.01096234  0.05134124  0.02810069
   0.03770116  0.0553779   0.57141093  0.0136704 ]
 [ 0.00385632  0.13782755  0.03619274  0.02080084  0.02646481  0.02299453
   0.02453488  0.6214677   0.05838575  0.04125445]
 [ 0.01262234  0.73579553  0.06984985  0.01125391  0.02800177  0.00856313
   0.01330206  0.04851836  0.04816946  0.01572

In [22]:
def get_values(l):
    #print (l)
    g=[0]*len(category_list)
    for i in l:
        g[i[0]]=i[1]
        
    return g
import collections

label_dict={}

def predict_label(doc_path):
    test_doc = [tokens for tokens in iter_wiki(doc_path)]
    #part = [get_values(lda_model[id2word_wiki.doc2bow(tokens)]) for tokens in test_doc] 
    labels=[km.predict([get_values(lda_model[id2word_wiki.doc2bow(tokens)])])[0] for tokens in test_doc]
    counter = collections.Counter(labels)
    print(counter)
    return counter.most_common(1)[0][0]



for path, subdirs, files in os.walk(root_folder):
    del subdirs[:]
    part=[]
    for file in files:
        if file.endswith('.xml'):
            doc_path = path + file
            print(doc_path)
            counter = predict_label(doc_path)
            
            #_label_dict[file.replace(".xml","")] = counter
            label_dict[file.replace(".xml","")] = counter
            
    
    
print(label_dict)


./AGHHLMMNRT/Arts.xml
Counter({3: 453, 5: 376, 2: 335, 8: 90, 6: 61, 9: 32, 4: 22, 0: 13, 1: 6})
./AGHHLMMNRT/Geography.xml
Counter({6: 273, 8: 146, 3: 137, 2: 118, 7: 90, 5: 75, 0: 66, 9: 48, 4: 20, 1: 5})
./AGHHLMMNRT/Health.xml
Counter({4: 1479, 2: 293, 3: 107, 5: 104, 8: 79, 9: 56, 6: 48, 0: 27, 7: 14, 1: 7})
./AGHHLMMNRT/History.xml
Counter({3: 373, 8: 316, 2: 283, 5: 212, 4: 134, 6: 131, 0: 8, 1: 7, 9: 3, 7: 1})
./AGHHLMMNRT/Literature.xml
Counter({5: 641, 2: 534, 3: 265, 8: 209, 4: 79, 0: 28, 6: 10, 1: 4, 9: 2})
./AGHHLMMNRT/Mathematics.xml
Counter({0: 364, 2: 213, 5: 111, 4: 94, 3: 82, 9: 75, 8: 56, 7: 46, 1: 8, 6: 5})
./AGHHLMMNRT/Music.xml
Counter({3: 378, 2: 335, 0: 226, 5: 218, 8: 96, 4: 34, 1: 29, 6: 13, 7: 9, 9: 6})
./AGHHLMMNRT/Nature.xml
Counter({4: 118, 0: 92, 3: 69, 9: 59, 2: 46, 6: 46, 8: 32, 5: 16, 7: 3, 1: 1})
./AGHHLMMNRT/Religion.xml
Counter({4: 251, 2: 247, 3: 181, 8: 82, 5: 62, 6: 11, 0: 7, 1: 1})
./AGHHLMMNRT/Technology.xml
Counter({5: 382, 7: 338, 1: 213, 0: 

In [ ]:
def drawgraph(x_label,y,file,text_data):
    
    import matplotlib.pyplot as plt

    x = np.arange(len(x_label))  # the x locations for the groups
    width = 0.3       # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x, y, width, color='blue')
    ax.set_ylabel('Weights')
    ax.set_title('Topic Distribution')
    #ax.set_xticks(x + width / 2)
    #ax.set_rotation(90)
    ax.set_xticklabels(category_list)
    ax.text(3, 8, text_data, style='italic',
        bbox={'facecolor':'green', 'alpha':0.5, 'pad':10})

    def autolabel(rects):
        """
        Attach a text label above each bar displaying its height
        """
        for rect in rects:
            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                    '%d' % int(height),
                    ha='center', va='bottom')

    autolabel(rects1)
    plt.xticks(rotation=90)
    #plt.setp(plt.xticks()[0], rotation=45)
    #fig = plt.figure(figuresize=4, 5)
    plt.savefig(file.replace(".xml","")+'.png')
    plt.close(fig)

In [23]:
# Test 
def getPart(testFile):
    test_doc = [tokens for tokens in iter_wiki(testFile)]
    part = [get_values(lda_model[id2word_wiki.doc2bow(tokens)]) for tokens in test_doc]
    return part


for path, subdirs, files in os.walk(testFolder):
    for file in files:
        if file.endswith('.xml'):
            doc_path = testFolder + file
            print(doc_path)
            
            part=getPart(doc_path)
            print(part)
            
            print(km.predict(part))
            
    

./Simplex1/test/Test_doc_AI.xml
[[0, 0, 0, 0, 0.75216177474420698, 0.23792168289009072, 0, 0, 0, 0]]
[0]
./Simplex1/test/Test_doc_Allegory.xml
[[0, 0.44688787122227502, 0.051161283057231145, 0, 0.011893890673878962, 0, 0, 0.11826688629582272, 0.37083911222211074, 0]]
[2]
./Simplex1/test/Test_doc_CS.xml
[[0, 0.38739112264163889, 0, 0.013455103442951186, 0.097264129072204913, 0, 0, 0.33947677866437376, 0, 0.16203058686206484]]
[2]


In [56]:
#Cosine Similarity

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer( max_features=500, stop_words='english')
t1=[]
with open("./Simplex1/test/Test_doc_AI.xml",encoding="UTF-8") as f:
    t1.append(str(f.read()))
t2=[]
with open("./Simplex1/test/Test_doc_Allegory.xml",encoding="UTF-8") as f:
    t1.append(str(f.read()))
    
print(len(t1))
#print(t1,t2)

#file = open("./Simplex1/test/Test_doc_AI.xml", 'r',encoding="UTF-8")
#p1=vectorizer.fit_transform(file.read())
#file = open("./Simplex1/test/Test_doc_CS.xml", 'r',encoding="UTF-8")
#p2=vectorizer.fit_transform(file.read())
p1=vectorizer.fit_transform(t1)
print(p1.shape)

from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(p1[0:1], p1))


#print(gensim.matutils.cossim(p1, p2))

2
(2, 500)
[[ 1.        0.253986]]
